<a href="https://colab.research.google.com/github/kauser-husainee/standardise-colors-scripts/blob/main/Copy_of_GENERAL_Standardise_Color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Standardise all to hex**

In [ ]:
import json

def rgb_to_hex(rgb_value):
    # Extract the numeric values from the RGB string
    rgb = rgb_value.strip('rgb() ').split(',')
    r, g, b = [int(component) for component in rgb]

    # Convert the RGB values to hexadecimal
    hex_value = '#{:02x}{:02x}{:02x}'.format(r, g, b)

    return hex_value

def rgba_to_hex(rgba_value):
    # Extract the numeric values from the RGBA string
    rgba = rgba_value.strip('rgba() ').split(',')
    r, g, b = [int(component) for component in rgba[:3]]

    # Convert the RGB values to hexadecimal
    hex_value = '#{:02x}{:02x}{:02x}'.format(r, g, b)

    # If there's an alpha component, add it to the hexadecimal value
    if len(rgba) == 4:
        a = float(rgba[3])
        alpha = int(a * 255)
        hex_value += '{:02x}'.format(alpha)

    return hex_value

# Parse the JSON data
with open('/content/colors.json') as json_file:
    colors_data = json.load(json_file)

# Create a new dictionary to store the updated colors
updated_colors = {}

# Iterate through each color name and value in the JSON
for color_name, color_value in colors_data.items():
    if 'rgba' in color_value:
        rgba_value = color_value
        hex_value = rgba_to_hex(rgba_value)
        updated_colors[color_name] = hex_value
    elif 'rgb' in color_value:
        rgb_value = color_value
        hex_value = rgb_to_hex(rgb_value)
        updated_colors[color_name] = hex_value
    else:
        updated_colors[color_name] = color_value

# Print the updated JSON
updated_json = json.dumps(updated_colors, indent=4)
with open('updated_colors.json', 'w') as updated_json_file:
    json.dump(updated_colors, updated_json_file, indent=4)


# **Sort all Hex**

In [ ]:
pip install colormath

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for colormath: filename=colormath-3.0.0-py3-none-any.whl size=39409 sha256=2550e6b8d700e97a780eb81d70705a16ee61a99fe14397394f601a423e63f649
  Stored in directory: /root/.cache/pip/wheels/ab/b3/4d/c0738759c25a1df01958068f162cf2a9dc3ab1da8b972cfcfc
Successfully built colormath


In [ ]:
import json
import webcolors
from colormath.color_objects import LabColor
from colormath.color_diff import delta_e_cie2000

# Read hex codes from JSON file
with open('reduced_colors.json', 'r') as file:
    hex_codes = json.load(file)

# Convert hex codes to Lab color space and calculate Delta E
colors = []
reference_color = LabColor(lab_l=100, lab_a=0, lab_b=0)  # Set a reference color for Delta E comparison
for key, hex_code in hex_codes.items():
    try:
        rgb = webcolors.hex_to_rgb(hex_code)
        lab_color = LabColor(rgb[0] / 255, rgb[1] / 255, rgb[2] / 255)
        delta_e = delta_e_cie2000(reference_color, lab_color)
        colors.append((delta_e, key, hex_code))
    except ValueError:
        # Ignore invalid hex color values
        pass

# Sort the colors based on Delta E values
sorted_colors = sorted(colors, key=lambda x: x[0])

# Extract sorted hex codes with original key names
sorted_hex_codes = {color[1]: color[2] for color in sorted_colors}

# Write sorted hex codes to JSON file
with open('sorted_colors.json', 'w') as file:
    json.dump(sorted_hex_codes, file, indent=4)


FileNotFoundError: ignored

# ***Cluster colors while replacing duplicates hex value only with reduced values***

In [ ]:
import json
from sklearn.cluster import KMeans
import numpy as np

def hex_to_rgb(hex):
    hex = hex.strip().lstrip('#')
    if len(hex) == 3:  # short form, duplicates each digit
        hex = ''.join([ch * 2 for ch in hex])
    elif len(hex) == 4:  # short form with alpha, duplicates each digit
        hex = ''.join([ch * 2 for ch in hex])
    elif len(hex) < 6:  # If less than 6, pad with zeros
        hex = hex.ljust(6, '0')
    elif len(hex) < 8 and len(hex) > 6:  # If between 6 and 8, pad with zeros
        hex = hex.ljust(8, '0')
    elif len(hex) != 6 and len(hex) != 8:
        raise ValueError(f"Invalid color code: {hex}")
    rgb = tuple(int(hex[i:i+2], 16) for i in range(0, len(hex), 2))
    return rgb[:3]  # Return RGB values only

def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

# Read JSON from a file
with open('updated_colors.json', 'r') as f:
    data = json.load(f)

# Convert the hex colors to RGB
colors_rgb = [hex_to_rgb(item) for item in data.values()]

# Define reduction factor
reduction_factor = 0.2  # Change this to control the degree of color clustering
n_clusters = max(1, int(len(set(colors_rgb)) * reduction_factor))

# Apply KMeans clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(colors_rgb)

# Map the original color names to their corresponding reduced color
reduced_colors = {color_name: rgb_to_hex([int(val) for val in kmeans.cluster_centers_[label]]) for color_name, label in zip(data.keys(), kmeans.labels_)}

# Find the removed colors by subtracting the reduced colors from the original colors
removed_colors = {key: value for key, value in data.items() if value not in reduced_colors.values()}

# Write reduced colors to a file
with open('SAMEFILE_reduced_colors.json', 'w') as f:
    json.dump(reduced_colors, f, indent=4)

# Write removed colors to a file
with open('SAMEFILE_removed_colors.json', 'w') as f:
    json.dump(removed_colors, f, indent=4)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**Clustered** with removed duplicated & a mapping file for removed colors

In [ ]:
import json
from sklearn.cluster import KMeans
import numpy as np
from collections import defaultdict

def hex_to_rgb(hex):
    hex = hex.strip().lstrip('#')
    if len(hex) == 3:  # short form, duplicates each digit
        hex = ''.join([ch * 2 for ch in hex])
    elif len(hex) == 4:  # short form with alpha, duplicates each digit
        hex = ''.join([ch * 2 for ch in hex])
    elif len(hex) < 6:  # If less than 6, pad with zeros
        hex = hex.ljust(6, '0')
    elif len(hex) < 8 and len(hex) > 6:  # If between 6 and 8, pad with zeros
        hex = hex.ljust(8, '0')
    elif len(hex) != 6 and len(hex) != 8:
        raise ValueError(f"Invalid color code: {hex}")
    rgb = tuple(int(hex[i:i+2], 16) for i in range(0, len(hex), 2))
    return rgb[:3]  # Return RGB values only

def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

# Read JSON from a file
with open('updated_colors.json', 'r') as f:
    data = json.load(f)

# Convert the hex colors to RGB
colors_rgb = [hex_to_rgb(item) for item in data.values()]

# Define reduction factor
reduction_factor = 0.1  # Change this to control the degree of color clustering
n_clusters = max(1, int(len(set(colors_rgb)) * reduction_factor))

# Apply KMeans clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(colors_rgb)

# Initialize an empty dictionary to store color clusters
color_clusters = defaultdict(list)

# Populate color_clusters with color names
for color_name, label in zip(data.keys(), kmeans.labels_):
    reduced_color_hex = rgb_to_hex([int(val) for val in kmeans.cluster_centers_[label]])
    color_clusters[reduced_color_hex].append(color_name)

# Prepare output data with reduced colors (the first color name in each cluster)
reduced_colors = {color_names[0]: color_hex for color_hex, color_names in color_clusters.items()}

# Prepare output data with removed colors (the remaining color names in each cluster)
removed_colors = {color_name: color_hex for color_hex, color_names in color_clusters.items() for color_name in color_names[1:]}

# Prepare output data with color mapping (mapping of removed colors to their corresponding reduced color)
color_mapping = {color_name: color_hex for color_hex, color_names in color_clusters.items() for color_name in color_names}

# Write reduced colors to a file
with open('reduced_colors.json', 'w') as f:
    json.dump(reduced_colors, f, indent=4)

# Write removed colors to a file
with open('removed_colors.json', 'w') as f:
    json.dump(removed_colors, f, indent=4)

# Write color mapping to a file
with open('color_mapping.json', 'w') as f:
    json.dump(color_mapping, f, indent=4)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
